<a href="https://colab.research.google.com/github/arunshaju001/Crime-Prediction-using-Machine-Learning/blob/testing/Project_crime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Crime Prediction and Visualization**

## **Libraries**

In [1]:
#Libraries for map 
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
from mpl_toolkits.basemap import Basemap

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 73.1 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgeos-dev amd64 3.6.2-1build2 [73.1 kB]
Fetched 73.1 kB in 1s (123 kB/s)
Selecting previously unselected package libgeos-dev.
(Reading database ... 144465 files and directories currently installed.)
Preparing to unpack .../libgeos-dev_3.6.2-1build2_amd64.deb ...
Unpacking libgeos-dev (3.6.2-1build2) ...
Setting up libgeos-dev (3.6.2-1build2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
     |████████████████████████████████| 133.1

In [2]:
!pip install xgboost
!pip install --upgrade xgboost

     |████████████████████████████████| 127.6MB 24kB/s 
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [3]:
import pandas as pd
import numpy as np
import datetime
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn import metrics 
import json
import matplotlib as mpl
import folium
from folium.plugins import MarkerCluster
import math
import matplotlib.pyplot as plt
from sklearn.externals import joblib 
import IPython
from google.colab import output
from IPython.display import clear_output
from xgboost import XGBClassifier
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


## **Mapping Lattitudes and Longitudes to Grids** - No need to Run!!!

In [4]:
#Creating dataset with grid value
# df.to_csv('/content/drive/My Drive/Colab Notebooks/Project/grid_data_large.csv', index = False)

In [5]:
#Rmoving rows with offset values
# df=df[df.Y <= 37.811127]
# df=df[ df.Y>=37.708318]
# df=df[df.X <= -122.356992]
# df=df[ df.X>=-122.514985]

In [6]:
#Lat Long Mapping
#Dont run!!!

# def get_grid_val(lat_x,lon_y):
#   lat_x=37.811128-lat_x
#   lat_x=lat_x/.0102809
#   lat_x=math.floor(lat_x)
#   lon_y=-122.514985-lon_y
#   lon_y=lon_y/.0157993
#   lon_y=abs(lon_y)
#   lon_y=math.floor(lon_y)
#   xy=lon_y*10+lat_x
#   return(xy)

# d=[]
# for index, row in df.iterrows(): 
#     d.append(get_grid_val(row['Y'], row['X']))
# df["grid"]=d
# print("success")

## **Preparing Dataset for Training**

**Read dataset from drive and write into Local**

In [7]:
#Read dataset from drive and write into sample_data
#sfnew - orginal
#grid_data - small with grid
#train - small ds
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project/grid_data_large.csv')
#Saving to local
data.to_csv('/content/sample_data/sfnew.csv', index=False)
#Reading the Dataset
df=pd.read_csv('/content/sample_data/sfnew.csv')
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,grid
0,146196161,NON-CRIMINAL,LOST PROPERTY,Tuesday,09/23/2014,01:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,POINT (-122.403404791479 37.775420706711),14619616171000,73
1,150045675,ASSAULT,BATTERY,Thursday,01/15/2015,17:00,TARAVAL,NONE,1800 Block of VICENTE ST,-122.485604,37.738821,POINT (-122.48560378101 37.7388214326705),15004567504134,17
2,140632022,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Wednesday,07/30/2014,09:32,BAYVIEW,NONE,100 Block of GILLETTE AV,-122.396535,37.710660,POINT (-122.396535107224 37.7106603302503),14063202264085,79
3,150383259,ASSAULT,BATTERY,Saturday,05/02/2015,23:10,BAYVIEW,"ARREST, BOOKED",2400 Block of PHELPS ST,-122.400131,37.730093,POINT (-122.400130573297 37.7300925390327),15038325904134,77
4,40855122,SUICIDE,SUICIDE BY JUMPING,Tuesday,07/27/2004,15:19,SOUTHERN,NONE,500 Block of I-80,-122.386667,37.789882,POINT (-122.386667033903 37.7898821569191),4085512260170,82


In [8]:
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,grid
0,146196161,NON-CRIMINAL,LOST PROPERTY,Tuesday,09/23/2014,01:00,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403405,37.775421,POINT (-122.403404791479 37.775420706711),14619616171000,73
1,150045675,ASSAULT,BATTERY,Thursday,01/15/2015,17:00,TARAVAL,NONE,1800 Block of VICENTE ST,-122.485604,37.738821,POINT (-122.48560378101 37.7388214326705),15004567504134,17
2,140632022,SUSPICIOUS OCC,INVESTIGATIVE DETENTION,Wednesday,07/30/2014,09:32,BAYVIEW,NONE,100 Block of GILLETTE AV,-122.396535,37.710660,POINT (-122.396535107224 37.7106603302503),14063202264085,79
3,150383259,ASSAULT,BATTERY,Saturday,05/02/2015,23:10,BAYVIEW,"ARREST, BOOKED",2400 Block of PHELPS ST,-122.400131,37.730093,POINT (-122.400130573297 37.7300925390327),15038325904134,77
4,40855122,SUICIDE,SUICIDE BY JUMPING,Tuesday,07/27/2004,15:19,SOUTHERN,NONE,500 Block of I-80,-122.386667,37.789882,POINT (-122.386667033903 37.7898821569191),4085512260170,82


**Creating year, month, day and hour columns**

In [9]:
#Creating year, month and day columns
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
# df['hour'] = pd.DatetimeIndex(df['Date']).hour  #In crime,grid_data
df['hour'] = pd.DatetimeIndex(df['Time']).hour    #In sfnew,grid_data_large

**Category conversion**

In [10]:
#Category conversion
crime_dict={  'WARRANTS':1,   #c7000a
              'LARCENY/THEFT':2, 'VEHICLE THEFT':2,'ROBBERY':2,'STOLEN PROPERTY':2,'EXTORTION':2, 'EMBEZZLEMENT':2,   #ed0274
              'VANDALISM':3,'ARSON':3,    #a6039e
              'ASSAULT':4, #830fa6
              'WEAPON LAWS':5,  #6b09eb
              'BURGLARY':6,'TRESPASS':6,    #171ee3          
              'MISSING PERSON':7,  'KIDNAPPING':7,    #2e86f2
              'SEX OFFENSES NON FORCIBLE':8, 'PROSTITUTION':8,'SEX OFFENSES, NON FORCIBLE':8,   #00b6c7
              'SEX OFFENSES FORCIBLE':9, 'SEX OFFENSES, FORCIBLE':9,   #00c25e            
              'LOITERING':10, 'SUICIDE':10,'DISORDERLY CONDUCT':10, 'DRIVING UNDER THE INFLUENCE':10, #none           
              'TREA':10, 'RECOVERED VEHICLE':10, 'OTHER OFFENSES':10,'BAD CHECKS':10,'GAMBLING':10,
              'FORGERY/COUNTERFEITING':10,'SECONDARY CODES':10,'NON-CRIMINAL':10,'SUSPICIOUS OCC':10, 'DRUG/NARCOTIC':10,  
              'FRAUD':10, 'RUNAWAY':10,'DRUNKENNESS':10, 'FAMILY OFFENSES':10,'LIQUOR LAWS':10,  'BRIBERY':10,        
              'PORNOGRAPHY/OBSCENE MAT':11  #c27e00 
              }
df["Category"] = df["Category"].replace(crime_dict)

**Day of weeks conversion**

In [11]:
#Day of weeks conversion
data_week_dict = { "Monday": 1, "Tuesday":2, "Wednesday":3, "Thursday":4, "Friday":5, "Saturday":6, "Sunday":7 }
df["DayOfWeek"] = df["DayOfWeek"].replace(data_week_dict)

 **District conversion**

In [12]:
#District conversion
district = df["PdDistrict"].unique()
data_dict_district = {}
count = 1
for data in district:
    data_dict_district[data] = count
    count+=1 
df["PdDistrict"] = df["PdDistrict"].replace(data_dict_district)

**Checking Data**

In [13]:
#Checking correlation
corr = df.corr()
print(corr["Category"])

IncidntNum   -0.039038
Category      1.000000
DayOfWeek    -0.026793
PdDistrict    0.038569
X             0.006231
Y            -0.046726
PdId         -0.039038
grid          0.017282
year         -0.031345
month        -0.014189
day          -0.020276
hour         -0.071228
Name: Category, dtype: float64


In [14]:
#Checking data distribution
skew = df.skew()
print(skew)

IncidntNum    0.705717
Category      0.078545
DayOfWeek    -0.008013
PdDistrict    0.093789
X            -1.208264
Y            -0.725909
PdId          0.705717
grid         -1.040506
year         -0.030696
month         0.021918
day           0.008271
hour         -0.516175
dtype: float64


## **Feature Selection and Creation of Prediction Dataset**

**Selecting features and Splitting Dataset**

In [15]:
#Selecting features
# "DayOfWeek" "PdDistrict" "X" "Y" "split" "grid" "year" "month" "day" "hour"
#All    "grid","year","DayOfWeek" ,"month","PdDistrict","day"
features = [  "grid","year","DayOfWeek" ,"month","PdDistrict","day" ]
df_new=df[features]
df_res=df.Category
#Splitting the data
train_X, test_X, train_y, test_y = train_test_split(df_new, df_res, train_size=0.9 , random_state = 1)

**Creating prediction_set for Predicting Crime**

In [16]:
# Creating prediction_set for Predicting Crime
columns=[ "grid","year","DayOfWeek" ,"month","PdDistrict","day" ]
pd.DataFrame(index=np.arange(1), columns=np.arange(8))
prediction_set = pd.DataFrame( columns=columns)
prediction_set = prediction_set.fillna(0)

#District assignment to Grid
grid_Pd_dict ={ 2: 6, 3: 6, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 12: 6, 13: 6, 14: 2, 15: 2, 16: 2, 17: 2, 18: 2, 19: 2, 20: 4,
                22: 6, 23: 6, 24: 2, 25: 2, 26: 2, 27: 2, 28: 2, 29: 2, 30: 8, 31: 6, 32: 6, 33: 10, 34: 2, 35: 10,
                36: 2, 37: 2, 38: 2, 39: 2, 40: 4, 41: 4, 42: 6, 43: 10, 44: 10, 45: 10, 46: 5, 47: 1, 48: 2,
                49: 2, 50: 4, 51: 4, 52: 4, 53: 4, 54: 10, 55: 7, 56: 5, 57: 5, 58: 5, 59: 5, 60: 8, 61: 8, 62: 8,
                63: 1, 64: 7, 65: 7, 66: 3, 67: 3, 68: 3, 69: 5, 70: 8, 71: 1, 72: 8, 73: 1, 74: 3, 75: 3, 76: 3,
                77: 3, 78: 3, 79: 3, 82: 1, 83: 3, 84: 3, 85: 3, 86: 3, 87: 3, 88: 3, 89: 3, 90: 1, 97: 3, 98: 3}

#Grid Correction
grid_val={    0 : 90, 10 : 91, 20 : 92, 30 : 93, 40 : 94, 50 : 95, 60 : 96, 70 : 97, 80 : 98, 90 : 99,
              1 : 80, 11 : 81, 21 : 82, 31 : 83, 41 : 84, 51 : 85, 61 : 86, 71 : 87, 81 : 88, 91 : 89,
              2 : 70, 12 : 71, 22 : 72, 32 : 73, 42 : 74, 52 : 75, 62 : 76, 72 : 77, 82 : 78, 92 : 79,
              3 : 60, 13 : 61, 23 : 62, 33 : 63, 43 : 64, 53 : 65, 63 : 66, 73 : 67, 83 : 68, 93 : 69,
              4 : 50, 14 : 51, 24 : 52, 34 : 53, 44 : 54, 54 : 55, 64 : 56, 74 : 57, 84 : 58, 94 : 59,
              5 : 40, 15 : 41, 25 : 42, 35 : 43, 45 : 44, 55 : 45, 65 : 46, 75 : 47, 85 : 48, 95 : 49,
              6 : 30, 16 : 31, 26 : 32, 36 : 33, 46 : 34, 56 : 35, 66 : 36, 76 : 37, 86 : 38, 96 : 39,
              7 : 20, 17 : 21, 27 : 22, 37 : 23, 47 : 24, 57 : 25, 67 : 26, 77 : 27, 87 : 28, 97 : 29,
              8 : 10, 18 : 11, 28 : 12, 38 : 13, 48 : 14, 58 : 15, 68 : 16, 78 : 17, 88 : 18, 98 : 19,
              9 : 0, 19 : 1, 29 : 2, 39 : 3, 49 : 4, 59 : 5, 69 : 6, 79 : 7, 89 : 8, 99 : 9}

#Adding Data to prediction_set 
grid_set=[]
for i in df.grid.unique():
  grid_set.append(i)
prediction_set["grid"]=grid_set
prediction_set["PdDistrict"]=df["grid"].replace(grid_Pd_dict)
def set_date(week,day,month,year):
  prediction_set["year"]=year
  prediction_set["DayOfWeek"]=week
  prediction_set["month"]=month
  prediction_set["day"]=day
set_date(1,1,1,2020)

## **Model Training and Prediction Using XGBoost**

**XGBoost Model Training and Saving Model**




In [17]:
#XGBoost
# xgb = XGBClassifier()
# xgb.fit(train_X, train_y)
# predictions_xgb = xgb.predict(test_X)
# print("Success")

**Loading Saved Model**

In [18]:
# Load the model from the file 
xgb = joblib.load('/content/drive/My Drive/Colab Notebooks/Project/model.pt')
model=xgb

**XGBoost Accuracy on Test Dataset**

In [19]:
#XGBoost Accuracy
# print("XGBoost Accuracy :", metrics.accuracy_score(test_y, predictions_xgb)*100)

#XGBoost Accuracy : 81.78259560373381

**Predicted Values and Preparing values for Map**

In [20]:
#Predicted Values and Preparing values for Map
map_val_dict={}
def predict_crime():
  predicted_category = model.predict(prediction_set[features])
  # prediction_set["grid"]=prediction_set["grid"].replace(grid_val)
  grid_list = prediction_set["grid"].tolist()

  for i in range(len(grid_list)):
    map_val_dict[grid_list[i]]=predicted_category[i]

  #Adding missing grids in map_val_dict
  miss_grid=[0,1,9,10,11,21,80,81,91,92,93,94,95,96,99]
  for i in miss_grid:
    map_val_dict[i]=10
  # print(map_val_dict.keys())
# predict_crime()

## **Other Models** - No need to Run!!!

In [21]:
#KNN 
# knn_model = KNeighborsClassifier(n_neighbors=50,weights='distance',n_jobs=2)
# knn_model.fit(train_X, train_y)
# # Save the model as a pickle in a file 
# joblib.dump(knn_model, '/content/drive/My Drive/Colab Notebooks/Project/knn_model_50.pt') 
# print("Success")

In [22]:
#KNN Accuracy
# predictions_knn = knn.predict(test_X)
# print("KNN Accuracy :", metrics.accuracy_score(test_y, predictions_knn)*100)

# KNN Accuracy : 51.51674429867669 

In [23]:
#Logistic Regression
# log = LogisticRegression(max_iter=4000,n_jobs=2)
# log.fit(train_X, train_y)
# predictions_lr = log.predict(test_X)
# print("Success")

In [24]:
#Logistic Regression Accuracy
# print("Logistic Regression Accuracy :", metrics.accuracy_score(test_y, predictions_lr)*100)

# Logistic Regression Accuracy : 39.88164878599661 

In [25]:
#Gaussian Naive Bayes
# gnb = GaussianNB() 
# gnb.fit(train_X, train_y) 
# predictions_gnb = gnb.predict(test_X) 
# print("Success")

In [26]:
#Gaussian Naive Bayes Accuracy
# print("Gaussian Naive Bayes Accuracy :", metrics.accuracy_score(test_y, predictions_gnb)*100)

# Gaussian Naive Bayes Accuracy : 20.21050254093733

## **Map Visualization**

### **Grid Function for Map**

In [27]:
#Grid Function
def get_geojson_grid(upper_right, lower_left, n=10):

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]
    grid_index=0
    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right,
                            "name": grid_index,
                            "category": int(map_val_dict[grid_index]),
                            
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates]
                    
                }
            }
            grid_index+=1
            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)
    
    return all_boxes

### **Map Creation**



In [28]:
#Map with Grid
lower_left = [37.708318, -122.514985]
upper_right = [37.811127, -122.356992]
SF_COORDINATES = (37.76, -122.45)

def create_map():
  lower_left = [37.708318, -122.514985]
  upper_right = [37.811127, -122.356992]

  SF_COORDINATES = (37.76, -122.45)
  f = folium.Figure(width=600, height=400)
  m = folium.Map(location=SF_COORDINATES, zoom_start=12).add_to(f)  #Creating Map
  grid = get_geojson_grid(upper_right, lower_left , n=10)           #Grid Creation

  for i, geo_json in enumerate(grid):


      # color = plt.cm.Reds(i / len(grid))
      # color = mpl.colors.to_hex(color)
      blank_list=[49,59,69,79,89,99,98,88,80,90,91,81]  #Empty cells
      if(i in blank_list): #or geo_json['properties']['category']==10):    #For No Crime
        continue
      elif(geo_json['properties']['category']==1):    #WARRANTS
        gj1 = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#c7000a"   #red
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj1)
      elif(geo_json['properties']['category'] == 2):    #THEFT
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#ed0274"   #pink
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 3):    #ARSON
          gj = folium.GeoJson(geo_json,
                              style_function=lambda feature : {'fillColor': "#dff702"   #yellow
                              ,'color':"black"
                              , 'opacity':0.135,'weight': 1,'fillOpacity': .65,} 
                              ).add_to(m) 
          m.add_child(gj)
      elif(geo_json['properties']['category'] == 4):    #ASSAULT
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#830fa6"   #violet
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category']== 5):   #WEAPON LAWS
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#aeb0ab"   #gray
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.75,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 6):  #BURGLARY
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#171ee3 "    #blue
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 7):    #KIDNAPPING
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#75bfba"   #cyan
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 8):    #SEX OFFENSES
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#00b6c7"   #light blue
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 9):    #SEX OFFENSES FORCIBLE
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#00c25e"   #green
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      elif(geo_json['properties']['category'] == 11):   #OBSCENE
        gj = folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "#c27e00 "    #dark yellow
                            ,'color':"black"
                            , 'opacity':0.135,'weight': 1,'fillOpacity': 0.65,} 
                            ).add_to(m) 
        m.add_child(gj)
      else:
        gj2=folium.GeoJson(geo_json,
                            style_function=lambda feature : {'fillColor': "black" 
                            ,'color':"black"
                            , 'opacity':0,'weight': .01,'fillOpacity': 0.05,} 
                            )
        m.add_child(gj2) 
    # folium.Marker([37.775421		,-122.403405	], popup= "here" ,  icon=folium.Icon(color='red')).add_to(m)
  display(m)  
set_date(1,31,10,2010)   #Set the values in the prediction_set
predict_crime()                 #Predict crime using the values
create_map()


## **Map in Html**

In [29]:
#UI for map with diffrent Predictions

def show_html():
  display(IPython.display.HTML('''
  <!-- CSS -->
    <style>
        button {  background-color: #0aa326;
                  border: none;
                  color: white;
                  padding: 10px 50px;
                  text-align: center;
                  text-decoration: none;
                  display: inline-block;
                  font-size: 16px;
                  margin: 4px 2px;
                  transition-duration: 0.4s;
                  cursor: pointer;
                  font-weight: bold;}
        button:hover {  background-color: #0aa326;
                        color: black; } 
        body { height: 500px; }
    </style>
    <style>
      /*the container must be positioned relative:*/
      .custom-select {
        position: relative;
        font-family: Arial;
      }
      .custom-select select {
        display: none; /*hide original SELECT element:*/
      }

      .select-selected {
        background-color: #0aa326;
      }

      /*style the arrow inside the select element:*/
      .select-selected:after {
        position: absolute;
        content: "";
        top: 14px;
        right: 10px;
        width: 0;
        height: 0;
        border: 6px solid transparent;
        border-color: #fff transparent transparent transparent;
      }

      /*point the arrow upwards when the select box is open (active):*/
      .select-selected.select-arrow-active:after {
        border-color: transparent transparent #fff transparent;
        top: 7px;
      }

      /*style the items (options), including the selected item:*/
      .select-items div,.select-selected {
        color: white;
        font-weight:bold;
        padding: 8px 16px;
        border: 1px solid transparent;
        border-color: transparent transparent rgba(0, 0, 0, 0.1) transparent;
        cursor: pointer;
        user-select: none;
      }

      /*style items (options):*/
      .select-items {
        position: absolute;
        background-color: #0aa326;
        top: 100%;
        left: 0;
        right: 0;
        z-index: 99;
      }

      /*hide the items when the select box is closed:*/
      .select-hide {
        display: none;
      }

      .select-items div:hover, .same-as-selected {
        background-color: #0aa326;
        color: black;
      }
      .dot {
        height: 15px;
        width: 15px;
        background-color: #bbb;
        border-radius: 50%;
        display: inline-block;
      }
    </style>
  <!-- body -->
    <body >
        <h1 ><center>Crime Prediction and Visualization</center></h1>
        <br><br><center>
        <div style="display: inline-flex; ">
          <div class="custom-select" style="width:150px; ">
            <select id="dayofweek">
              <option value="0">Week Day</option>
              <option value="1">Monday</option> 
              <option value="2">Tuesday</option> 
              <option value="3">Wednesday</option>
              <option value="4">Thursday</option>
              <option value="5">Friday</option>
              <option value="6">Saturday</option>
              <option value="7">Sunday</option>
            </select>
          </div>
          <div class="custom-select" style="width:150px; padding-inline-start: 40px;">
            <select id="day">
              <option value="0">Day</option>
              <option value="1">1</option>
              <option value="2">2</option>
              <option value="3">3</option>
              <option value="4">4</option>
              <option value="5">5</option>
               <option value="6">6</option>
                <option value="7">7</option>
                <option value="8">8</option>
                <option value="9">9</option>
                <option value="10">10</option>
                <option value="11">11</option>
                <option value="12">12</option>
                <option value="13">13</option>
                <option value="14">14</option>
                <option value="15">15</option>
                <option value="16">16</option>
                <option value="17">17</option>
                <option value="18">18</option>
                <option value="19">19</option>
                <option value="20">20</option>
                <option value="21">21</option>
                <option value="22">22</option>
                <option value="23">23</option>
                <option value="24">24</option>
                <option value="25">25</option>
                <option value="26">26</option>
                <option value="27">27</option>
                <option value="28">28</option>
                <option value="29">29</option>
                <option value="30">30</option>
                <option value="31">31</option>
            </select>
          </div>
          <div class="custom-select" style="width:150px; padding-inline-start: 40px;">
            <select id="month">
              <option value="0">Month</option>
              <option value="1">January</option>
              <option value="2">February</option>
              <option value="3">March</option>
              <option value="4">April</option>
              <option value="5">May</option>
              <option value="6">June</option>
              <option value="7">July</option>
              <option value="8">August</option>
              <option value="9">September</option>
              <option value="10">October</option>
              <option value="11">November</option>
              <option value="12">December</option>
            </select>
          </div>
          <div class="custom-select" style="width:150px; padding-inline-start: 40px;">
            <select id="year">
              <option value="0">Year</option>
              <option value="2010">2010</option>
              <option value="2011">2011</option>
              <option value="2012">2012</option>
              <option value="2013">2013</option>
              <option value="2014">2014</option>
              <option value="2015">2015</option>
              <option value="2016">2016</option>
              <option value="2017">2017</option>
              <option value="2018">2018</option>
              <option value="2019">2019</option>
              <option value="2020">2020</option>
              <option value="2021">2021</option>
            </select>
          </div>
        </div>
        <button id='predict_button' style="margin-left:40px;" >Predict</button></center>
        <div id="text" style="text-align:center"></div><br>
        <div style="text-align:center">
          <span class="dot"  style="background-color: #c7000a"></span>&nbsp;WARRANTS  &nbsp; 
          <span class="dot"  style="background-color: #ed0274"></span>&nbsp;THEFT	&nbsp;
          <span class="dot"  style="background-color: #dff702"></span>&nbsp;ARSON	&nbsp;
          <span class="dot"  style="background-color: #830fa6"></span>&nbsp;ASSAULT	&nbsp;
          <span class="dot"  style="background-color: #aeb0ab"></span>&nbsp;WEAPON LAWS	&nbsp;
          <span class="dot"  style="background-color: #171ee3"></span>&nbsp;BURGLARY	&nbsp;
          <span class="dot"  style="background-color: #75bfba"></span>&nbsp;KIDNAPPING	&nbsp;
          <span class="dot"  style="background-color: #00b6c7"></span>&nbsp;SEX OFFENSES	&nbsp;
          <span class="dot"  style="background-color: #00c25e"></span>&nbsp;SEX OFFENSES FORCIBLE &nbsp;	
          <span class="dot"  style="background-color: #c27e00"></span>&nbsp;OBSCENE	&nbsp;
        </div>
  <!-- JS -->
    <script>       //Get selected items from list 
          document.querySelector('#predict_button').onclick = () => {
            var dayofweek = document.getElementById("dayofweek");
            var dayofweek_selected = dayofweek.options[dayofweek.selectedIndex].value;
            var day = document.getElementById("day");
            var day_selected = day.options[day.selectedIndex].value;
            var month = document.getElementById("month");
            var month_selected = month.options[month.selectedIndex].value;
            var year = document.getElementById("year");
            var year_selected = year.options[year.selectedIndex].value;
            // google.colab.kernel.invokeFunction('fn', [], {{}}) 
            invoke = google.colab.kernel.invokeFunction
            invoke('fn', [dayofweek_selected,day_selected,month_selected,year_selected], {})  //Python function call  *args, **kw

            //document.getElementById("text").innerHTML = dayofweek_selected+" "+day_selected;
          };
    </script>
    <script>   // for drop down 
      var x, i, j, selElmnt, a, b, c;
      /*look for any elements with the class "custom-select":*/
      x = document.getElementsByClassName("custom-select");
      for (i = 0; i < x.length; i++) {
        selElmnt = x[i].getElementsByTagName("select")[0];
        /*for each element, create a new DIV that will act as the selected item:*/
        a = document.createElement("DIV");
        a.setAttribute("class", "select-selected");
        a.innerHTML = selElmnt.options[selElmnt.selectedIndex].innerHTML;
        x[i].appendChild(a);
        /*for each element, create a new DIV that will contain the option list:*/
        b = document.createElement("DIV");
        b.setAttribute("class", "select-items select-hide");
        for (j = 1; j < selElmnt.length; j++) {
          /*for each option in the original select element,
          create a new DIV that will act as an option item:*/
          c = document.createElement("DIV");
          c.innerHTML = selElmnt.options[j].innerHTML;
          c.addEventListener("click", function(e) {
              /*when an item is clicked, update the original select box,
              and the selected item:*/
              var y, i, k, s, h;
              s = this.parentNode.parentNode.getElementsByTagName("select")[0];
              h = this.parentNode.previousSibling;
              for (i = 0; i < s.length; i++) {
                if (s.options[i].innerHTML == this.innerHTML) {
                  s.selectedIndex = i;
                  h.innerHTML = this.innerHTML;
                  y = this.parentNode.getElementsByClassName("same-as-selected");
                  for (k = 0; k < y.length; k++) {
                    y[k].removeAttribute("class");
                  }
                  this.setAttribute("class", "same-as-selected");
                  break;
                }
              }
              h.click();
          });
          b.appendChild(c);
        }
        x[i].appendChild(b);
        a.addEventListener("click", function(e) {
            /*when the select box is clicked, close any other select boxes,
            and open/close the current select box:*/
            e.stopPropagation();
            closeAllSelect(this);
            this.nextSibling.classList.toggle("select-hide");
            this.classList.toggle("select-arrow-active");
          });
      }
      function closeAllSelect(elmnt) {
        /*a function that will close all select boxes in the document,
        except the current select box:*/
        var x, y, i, arrNo = [];
        x = document.getElementsByClassName("select-items");
        y = document.getElementsByClassName("select-selected");
        for (i = 0; i < y.length; i++) {
          if (elmnt == y[i]) {
            arrNo.push(i)
          } else {
            y[i].classList.remove("select-arrow-active");
          }
        }
        for (i = 0; i < x.length; i++) {
          if (arrNo.indexOf(i)) {
            x[i].classList.add("select-hide");
          }
        }
      }
      /*if the user clicks anywhere outside the select box,
      then close all select boxes:*/
      document.addEventListener("click", closeAllSelect);
    </script>
  </body>  
  <!-- End -->
    '''))

def fn(week,day,month,year):
  clear_output(wait=False)
  set_date(week,day,month,year)   #Set the values in the prediction_set
  predict_crime()                 #Predict crime using the values
  show_html()                     #Display html elements
  print("\n\nWeek : "+str(week)+" , "+"day : "+str(day)+" , "+"month : "+str(month)+" , "+"year : "+str(year)+"\n\n",end='')
  create_map()                    #Create map with values
  # print(prediction_set.head(1) ) 
  # print(map_val_dict)                 
  # display(m)                      #Display Map
output.register_callback('fn', fn)
fn(1,1,1,2020)



Week : 0 , day : 0 , month : 0 , year : 0

